CONCATENING TEXT DATASETS, DON'T HAVE TO RUN THIS SECTION ANYMORE

In [47]:
#DONT RUN THIS ANYMORE

from datasets import load_dataset
from convokit import Corpus, download

ds_ag_news = load_dataset("fancyzhx/ag_news") #news, formal
ds_sms = load_dataset("ucirvine/sms_spam") #should only use ham labels, text messages, informal
ds_edu = load_dataset("Locutusque/UltraTextbooks") #synthetic and human written textbooks..... acdademic / formal
#https://huggingface.co/datasets/fabiochiu/medium-articles
ds_medium = load_dataset('csv', data_files='/Users/rheamadhogarhia/Desktop/Year 3/NLP/nlpfinal/medium_articles.csv') #medium articles, academic but also likely less formal ? community contributions
#https://huggingface.co/datasets/kariatouk/enron_curated_labeled/tree/main
ds_enron= load_dataset('csv', data_files='/Users/rheamadhogarhia/Desktop/Year 3/NLP/nlpfinal/enron_labeled_curated.csv') #enron, very formal business emails probably?


ds_podcast_news = load_dataset("Whispering-GPT/lex-fridman-podcast")
ds_telephone_convos = Corpus(filename=download("switchboard-corpus"))
ds_ted_talk_edu = load_dataset('csv', data_files="/Users/rheamadhogarhia/Desktop/Year 3/NLP/nlpfinal/ted_talks_en.csv")
ds_business_meetings = load_dataset("edinburghcstr/ami", "ihm")
ds_un_debates = load_dataset('csv', data_files="/Users/rheamadhogarhia/Desktop/Year 3/NLP/nlpfinal/un-general-debates.csv")

#this cell ran in 21 minutes (probably don't have to load the whole dataset?)



Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/91 [00:00<?, ?it/s]

In [52]:
#TRAIN SETS, choosing 500 samples frome each written text set
num_samples = 1000
ds_ag_news_sub = ds_ag_news["train"].shuffle(seed=42).select(range(num_samples))

#filtering out the spam label - only getting legitimate texts
ds_sms_filtered = ds_sms["train"].filter(lambda example: example['label'] == 0)
ds_sms_sub = ds_sms_filtered.shuffle(seed=42).select(range(num_samples))

ds_edu_sub = ds_edu["train"].shuffle(seed=42).select(range(num_samples))
ds_medium_sub = ds_medium["train"].shuffle(seed=42).select(range(num_samples))
ds_enron_sub = ds_enron["train"].shuffle(seed=42).select(range(num_samples))


In [54]:
#combine
from datasets import Dataset, concatenate_datasets
import random

ag_news_dataset = Dataset.from_dict({"text": ds_ag_news_sub["text"], 
                                     "source": ["ag_news"] * len(ds_ag_news_sub)})
medium_dataset = Dataset.from_dict({"text": ds_medium_sub["text"], 
                                    "source": ["medium"] * len(ds_medium_sub)})
edu_dataset = Dataset.from_dict({"text": ds_edu_sub["text"], 
                                 "source": ["textbooks"] * len(ds_edu_sub)})
sms_dataset = Dataset.from_dict({"text": ds_sms_sub["sms"], 
                                 "source": ["sms"] * len(ds_sms_sub)})
enron_dataset = Dataset.from_dict({"text": ds_enron_sub["email"], 
                                   "source": ["enron_emails"] * len(ds_enron_sub)})


combined_dataset = concatenate_datasets([ag_news_dataset, medium_dataset, edu_dataset, sms_dataset, enron_dataset])

combined_dataset = combined_dataset.add_column("type", ["written"] * len(combined_dataset))

#Shuffle
combined_dataset = combined_dataset.shuffle(seed=42)

combined_dataset.to_csv("shuffled_combined_data_with_type.csv")

print(combined_dataset[:5])


Creating CSV from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

{'text': ['Security designed to meet the needs of your business.\n\nToday, there are too many security controls and not enough talent to manage the complexity. We understand organizations may have limited time and resources; and security needs to be simple and end to end.\n\nAs an organization, your employees have left the building; you definitely need security controls that span across your offices and users wherever they are, whatever devices they are using, across the cloud, through the network and onto the endpoint.\n\nYour security controls also need to allow your employees to stay connected, protected and productive seamlessly.\n\nAt Signal Alliance, we often speak with businesses who believe that, because they are small, they aren’t interesting to the hacktivists and cyber criminals.\n\nActually, we see the reverse, often the smaller customers are a gateway to the bigger ones. Supply chain attacks like this are responsible for some of the biggest breaches that we’ve seen. The br

START RUNNING FROM HERE